In [1]:
import pandas as pd
import numpy as np

from pathlib import Path

In [2]:
# Machine-learning specific imports
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import model_from_json

import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler

%matplotlib inline

In [3]:
## API Imports 
import os
import requests
import json

from dotenv import load_dotenv
load_dotenv()

True

In [4]:
## Load pickle for exports and imports of data  
import pickle 
def load_obj(path):
    with open(path, 'rb') as f:
        return pickle.load(f)
    
def save_obj(obj, path ):
    with open(path, 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

In [5]:
## FMP Constants 
fmpbase_urlv3 = 'https://fmpcloud.io/api/v3/'
fmpbase_urlv4 = 'https://fmpcloud.io/api/v4/'
api_key = os.getenv("FMP_CLOUD_API_KEY")

start_date = '2022-01-01'
end_date = '2022-01-31'

## FMP Functions 
def get_FMP_historical_data(symbol, startDate=start_date, endDate=end_date, apiKey=api_key):
    url_hist_price = fmpbase_urlv3+'historical-price-full/'
    url_hist_query_with_date = url_hist_price+symbol+'?from='+startDate+'&to='+endDate+'&apikey='+apiKey
    resp_data = requests.get(url_hist_query_with_date)
    json_ = resp_data.json()
    data = json_['historical']
    df = pd.DataFrame(data)
    df.rename(columns={'date':'Date'},inplace=True)
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.reindex(index=df.index[::-1]) ## Reverse the DataFrame 
    df.set_index('Date',inplace=True)
    df.drop(columns='label',inplace=True)
    return df

In [6]:
def get_data(symbol, n_days):
    path = Path('../FilesExport_Complete_DFs_TI_noShift/'+symbol+'_TI_DF_no_shift.pkl')
    data = load_obj(path)
    df = data[symbol]
    
    df_close = df[['close']]
    df_close = df_close.reset_index().rename(columns={"Date": "Close_Date"})
    
    features_df = df.reset_index().drop(columns=['close','adjClose'])
    
    new_close_df = df_close.iloc[n_days: , :].reset_index(drop=True)
    
    ## Prevent multiple API calls each time, but use API when needed. 
    try:
        path = Path('../FilesExport_Updated_API_data/'+symbol+'_jan_2022.pkl')
        api_df = load_obj(path)
    except:
        api_df = get_FMP_historical_data(symbol)
    

    new_data = api_df[['close']]
    new_data = new_data.reset_index().rename(columns={"Date": "Close_Date"})
    new_data = new_data.iloc[0:n_days]
    
    new_close_df = new_close_df.append(new_data, ignore_index=True)
    

    
    return features_df, new_close_df

## Dropping QUANTITY_FAILS from dataframe before machine learning 

def prepare_data_no_FTD(symbol,n_days,return_data=False):
    features_df, new_close_df = get_data(symbol,n_days)
    
    X = features_df.drop(columns={'Date','QUANTITY_FAILS'}).values
    y = new_close_df['close'].values
    
    scaler = StandardScaler().fit(X)
    X = scaler.transform(X)
    
    if return_data == True:
        return X, y, features_df, new_close_df
    else:
        return X, y
    
def prepare_data(symbol,n_days,return_data=False):
    features_df, new_close_df = get_data(symbol,n_days)
    
    X = features_df.drop(columns='Date').values
    y = new_close_df['close'].values
    
    scaler = StandardScaler().fit(X)
    X = scaler.transform(X)
    
    if return_data == True:
        return X, y, features_df, new_close_df
    else:
        return X, y

In [7]:
## Make function that accepts a symbol, and returns 4 dataframes of predicted data, for each model
## that can then be used to make plots, etc. 

def get_prediction_dfs(symbol,n_days,include_model_obj=True):
    
    data_dict = {}
    
    # n_days = 10  ## Has to be 1, 2, 5, or 10 days. 
    # symbol = 'XRT'
    # days_to_plot = 60   ## 23 shows January Data

    export_path='../Model_Data/Date_Test_NN_w_FTD_all/High_Acc/'
    model_type = 'NN'
    n_days_string = str(n_days)
    export_path_prefix = export_path+symbol+'_'+model_type+'_'+n_days_string

    file_path = Path(export_path_prefix+'_model_data.json')
    with open(file_path, "r") as json_file:
        model_json = json_file.read()
    loaded_model1 = model_from_json(model_json)

    # load weights into new model
    file_path = Path(export_path_prefix+'_model_weights.h5')
    loaded_model1.load_weights(file_path)

    ## Load model summary 
    file_path = Path(export_path_prefix+'_model_summary.pkl')
    model_summary1 = load_obj(file_path)

    X1 , y1, features1, close_df1 = prepare_data(symbol,n_days,return_data=True)


    close_df1["predicted"] = loaded_model1.predict(X1)
    close_df1.set_index('Close_Date',inplace=True)

    prediction_df1 = close_df1[['close','predicted']]



    export_path='../Model_Data/Date_Test_NN_w_FTD_all/Low_Acc/'
    model_type = 'NN'
    n_days_string = str(n_days)
    export_path_prefix = export_path+symbol+'_'+model_type+'_'+n_days_string

    file_path = Path(export_path_prefix+'_model_data.json')
    with open(file_path, "r") as json_file:
        model_json = json_file.read()
    loaded_model2 = model_from_json(model_json)

    # load weights into new model
    file_path = Path(export_path_prefix+'_model_weights.h5')
    loaded_model2.load_weights(file_path)

    ## Load model summary 
    file_path = Path(export_path_prefix+'_model_summary.pkl')
    model_summary2 = load_obj(file_path)

    #X2 , y2, features2, close_df2 = prepare_data(symbol,n_days,return_data=True)

    close_df1["predicted"] = loaded_model2.predict(X1)
    #close_df.set_index('Close_Date',inplace=True) ## Alreayd set above 

    prediction_df2 = close_df1[['close','predicted']]


    export_path='../Model_Data/Date_Test_NN_noFTD_all/High_Acc/'
    model_type = 'NN'
    n_days_string = str(n_days)
    export_path_prefix = export_path+symbol+'_'+model_type+'_'+n_days_string

    file_path = Path(export_path_prefix+'_model_data.json')
    with open(file_path, "r") as json_file:
        model_json = json_file.read()
    loaded_model3 = model_from_json(model_json)

    # load weights into new model
    file_path = Path(export_path_prefix+'_model_weights.h5')
    loaded_model3.load_weights(file_path)

    ## Load model summary 
    file_path = Path(export_path_prefix+'_model_summary.pkl')
    model_summary3 = load_obj(file_path)

    X2 , y2, features2, close_df2 = prepare_data_no_FTD(symbol,n_days,return_data=True)


    close_df2["predicted"] = loaded_model3.predict(X2)
    close_df2.set_index('Close_Date',inplace=True)

    prediction_df3 = close_df2[['close','predicted']]



    export_path='../Model_Data/Date_Test_NN_noFTD_all/Low_Acc/'
    model_type = 'NN'
    n_days_string = str(n_days)
    export_path_prefix = export_path+symbol+'_'+model_type+'_'+n_days_string

    file_path = Path(export_path_prefix+'_model_data.json')
    with open(file_path, "r") as json_file:
        model_json = json_file.read()
    loaded_model4 = model_from_json(model_json)

    # load weights into new model
    file_path = Path(export_path_prefix+'_model_weights.h5')
    loaded_model4.load_weights(file_path)

    ## Load model summary 
    file_path = Path(export_path_prefix+'_model_summary.pkl')
    model_summary4 = load_obj(file_path)

    #X2 , y2, features2, close_df2 = prepare_data_no_FTD(symbol,n_days,return_data=True)

    close_df2["predicted"] = loaded_model4.predict(X2)
    #close_df2.set_index('Close_Date',inplace=True)

    prediction_df4 = close_df2[['close','predicted']]
    
    
    
    ## Instead of returning 4 dataframes, return one big dict containing all data needed 
    ## for one symbol 
    
    if include_model_obj == True:
    
        data_dict[symbol] = {
            'w_FTD': 
            {
                'model_1': 
                {
                    'loaded_model':loaded_model1,
                    'model_summary':model_summary1,
                    'prediction_df':prediction_df1,
                },
                'model_2': 
                {
                    'loaded_model':loaded_model2,
                    'model_summary':model_summary2,
                    'prediction_df':prediction_df2,
                }
            },
            'woFTD': 
            {
                'model_3': 
                {
                    'loaded_model':loaded_model3,
                    'model_summary':model_summary3,
                    'prediction_df':prediction_df3,
                },
                'model_4': 
                {
                    'loaded_model':loaded_model4,
                    'model_summary':model_summary4,
                    'prediction_df':prediction_df4,
                }
            }
        }
        
    elif include_model_obj==False:
        data_dict[symbol] = {
            'w_FTD': 
            {
                'model_1': 
                {
                    #'loaded_model':loaded_model1,
                    'model_summary':model_summary1,
                    'prediction_df':prediction_df1,
                },
                'model_2': 
                {
                    #'loaded_model':loaded_model2,
                    'model_summary':model_summary2,
                    'prediction_df':prediction_df2,
                }
            },
            'woFTD': 
            {
                'model_3': 
                {
                    #'loaded_model':loaded_model3,
                    'model_summary':model_summary3,
                    'prediction_df':prediction_df3,
                },
                'model_4': 
                {
                    #'loaded_model':loaded_model4,
                    'model_summary':model_summary4,
                    'prediction_df':prediction_df4,
                }
            }
        }
    
    
    
    
    return data_dict

In [9]:
## Create for loop that goes through 06_01_ML_symbol_success_list to find the most accurate models

In [10]:
## Import symbol list 
path = Path('../Resources/06_01_ML_symbol_success_list.pkl')
symbol_list = load_obj(path)
len(symbol_list)

770

In [43]:
# test_list = symbol_list[0:10]
# test_list

In [37]:
n_days = 10

acc_data_dict = {}
symbol_success_list = []
symbol_err_list = []

for symbol in symbol_list:
    try:
        data_dict = get_prediction_dfs(symbol,n_days,include_model_obj=False)

        
        acc_w_ftd_low = data_dict[symbol]['w_FTD']['model_2']['model_summary']['model_accuracy']
        acc_w_ftd_high= data_dict[symbol]['w_FTD']['model_1']['model_summary']['model_accuracy']

        acc_wo_ftd_low = data_dict[symbol]['woFTD']['model_4']['model_summary']['model_accuracy']
        acc_wo_ftd_high= data_dict[symbol]['woFTD']['model_3']['model_summary']['model_accuracy']
        
        ## Save acc's in larger dict to compare all symbols 
        acc_data_dict[symbol] = {
            'acc_w_ftd_low' : acc_w_ftd_low,
            'acc_w_ftd_high':acc_w_ftd_high,
            'acc_wo_ftd_low': acc_wo_ftd_low,
            'acc_wo_ftd_high':acc_wo_ftd_high,
        }
        
        symbol_success_list.append(symbol)
    except:
        symbol_err_list.append(symbol)
        continue   
#acc_data_dict

In [44]:
symbol_err_list

['EYEG', 'KBSF', 'KSU', 'MULN', 'SNDA']

In [39]:
acc_df = pd.DataFrame.from_dict(acc_data_dict,orient='index')
#acc_df

In [46]:
len(acc_df)

765

In [41]:
acc_df.sort_values('acc_w_ftd_low').head(20)

,acc_w_ftd_low,acc_w_ftd_high,acc_wo_ftd_low,acc_wo_ftd_high
XPL,0.002665,0.003246,0.002729,0.004247
DHY,0.003253,0.004764,0.004024,0.004893
AUMN,0.003963,0.004876,0.004049,0.006080
CPHI,0.004752,0.006047,0.005089,0.006426
UAMY,0.005359,0.006495,0.005740,0.006685
MSN,0.007210,0.008461,0.007300,0.008999
DNN,0.007564,0.016836,0.010779,0.015007
KIQ,0.008176,0.012833,0.008146,0.008901
VGZ,0.008192,0.012399,0.008193,0.009436
ITP,0.008715,0.011329,0.008977,0.010746


In [42]:
acc_df.sort_values('acc_wo_ftd_low').head(20)

,acc_w_ftd_low,acc_w_ftd_high,acc_wo_ftd_low,acc_wo_ftd_high
XPL,0.002665,0.003246,0.002729,0.004247
DHY,0.003253,0.004764,0.004024,0.004893
AUMN,0.003963,0.004876,0.004049,0.006080
CPHI,0.004752,0.006047,0.005089,0.006426
UAMY,0.005359,0.006495,0.005740,0.006685
MSN,0.007210,0.008461,0.007300,0.008999
KIQ,0.008176,0.012833,0.008146,0.008901
VGZ,0.008192,0.012399,0.008193,0.009436
ITP,0.008715,0.011329,0.008977,0.010746
AAU,0.009443,0.010728,0.009279,0.011833


In [45]:
## Export acc_df
path = Path('../Resources/date_shift_acc_df_post_ML.pkl')
save_obj(acc_df,path)